In [28]:
import quandl
import numpy as np
import types
import pandas as pd 
quandl.ApiConfig.api_key = "Xosv2zyeb3Dhzzx1g5nJ"
#data = quandl.get("CUR/CNY", returns="numpy")
cny = quandl.get("CUR/CNY", start_date="2016-01-01", end_date="2017-01-01", returns="numpy")
aud = quandl.get("CUR/AUD", start_date="2016-01-01", end_date="2017-01-01", returns="numpy")
cny_aud=cny[:].RATE/aud[:].RATE
dates=[]
# print(data2[2].dtype)

# df = pd.DataFrame(np_array)
# df.to_csv("file_path.csv")
for row in cny:
    dates.append(int(row[0].strftime('%Y%m%d')))
df = pd.DataFrame({"date" : dates, "rate" : cny_aud})
df.to_csv("cnyToaud.csv", index=False)    
#print(data2[['RATE']]/data[['RATE']])

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [30]:
import numpy as np
import pandas as pd 
import os
os.environ['KERAS_BACKEND']='theano'
import types
import quandl
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, TimeDistributed, Dropout
from keras.optimizers import Adam

TIME_STEPS = 6
BATCH_SIZE = 1
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 20
LR = 0.006

quandl.ApiConfig.api_key = "Xosv2zyeb3Dhzzx1g5nJ"
dates = []
prices = []

data = pd.read_csv('cnyToaud.csv')
dates=data['date'].tolist()
prices=data['rate'].tolist()
bench= np.ones(len(dates), dtype=np.int)*20160101
dates=dates-bench
# print(dates)
# print(prices)
def predict_price(dates, prices, x):
    dates = np.reshape(dates,(len(dates), 1)) # converting to matrix of n X 1
    svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1) # defining the support vector regression models
    svr_rbf.fit(dates, prices) # fitting the data points in the models
    return svr_rbf.predict(x)

def liner_predict(dates, prices, x):
    dates = np.reshape(dates,(len(dates), 1))
    model = LinearRegression()
    model.fit(dates, prices)
    return model.predict(x)

def nn_predict(dates, prices,x):
    dates=np.array(dates)
    prices=np.array(prices)
    x=np.array(x)
    X_train, Y_train = dates[:317], prices[:317]    
    X_test, Y_test = dates[317:], prices[317:]       
    # build a neural network from the 1st layer to the last layer
    model = Sequential()
    model.add(Dense(output_dim=10, input_dim=1))
#     model.add(Activation('tanh'))
#     model.add(Dense(output_dim=10, input_dim=10))
    model.add(Activation('tanh'))
    model.add(Dense(output_dim=1))
    # choose loss function and optimizing method
    model.compile(loss='mse', optimizer='adam') #sgd RMSprop Adam
    # training
    print('Training -----------')
    for step in range(10000):
        cost = model.train_on_batch(dates, prices)
        if step % 1000 == 0:
            print('train cost: ', cost)
    # test
#     print('\nTesting ------------')
#     cost = model.evaluate(X_test, Y_test, batch_size=50)
#     print('test cost:', cost)
#     W, b = model.layers[0].get_weights()
#     print('Weights=', W, '\nbiases=', b)
    return model.predict(x)
def get_batch(train):
    result = np.array(train)
    result = np.reshape(train[:6*50],(50, 6))
#     if normalise_window:
#         result = normalise_windows(result) 
#     row = round(0.9 * result.shape[0])
#     train = result[:int(row), :]
#     np.random.shuffle(train)
    x_train = result[:, :-1]
    y_train = result[:, -1]
# seq[:, :, np.newaxis], res[:, :, np.newaxis]
#     x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
#     x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  
#     return [x_train, y_train]
    return [x_train[:, :, np.newaxis], y_train]

def LSTM_predict( prices, x):
    model = Sequential()
    # build a LSTM RNN
#     model.add(LSTM(
#         batch_input_shape=(BATCH_SIZE, TIME_STEPS-1, INPUT_SIZE),
#         output_dim=CELL_SIZE,
#         return_sequences=False,      # True: output at all steps. False: output as last step.
#         stateful=True,              # True: the final state of batch1 is feed into the initial state of batch2
#     ))
#     # add output layer
# #     model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
#     model.add(Dense(OUTPUT_SIZE))
#     adam = Adam(LR)
#     model.compile(optimizer=adam,loss='mse')
    
    model.add(LSTM(
    input_dim=1,
    output_dim=50,
    return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(100,return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(output_dim=1))
    model.add(Activation('linear'))

    model.compile(loss='mse', optimizer='adam')
    print('Training ------------')
#     for step in range(501):
#         X_batch, Y_batch = get_batch(prices)
#         cost = model.train_on_batch(X_batch, Y_batch)
#         if step % 10 == 0:
#             print('train cost: ', cost)
    X_batch, Y_batch = get_batch(prices)
    model.fit(X_batch, Y_batch, nb_epoch=1, batch_size=1,)        
    x=np.array(x).reshape((BATCH_SIZE, TIME_STEPS-1, INPUT_SIZE))
    pred = model.predict(x,BATCH_SIZE)
    return pred
    
# predicted_price = predict_price(dates, prices, 20170404)  
# print(predicted_price)
# predicted_price = liner_predict(dates, prices, 20170404) 
# print(predicted_price)
# predicted_price = nn_predict(dates, prices,[911,10000,10303,10304,]) #20170404-20160101
# print(predicted_price)
predicted_price = LSTM_predict(prices,[10301,10302,10303,10304,10305]) #20170404-20160101
print(predicted_price)
last_data = quandl.get("CUR/CNY", rows=1, returns="numpy").RATE/quandl.get("CUR/AUD", rows=1, returns="numpy").RATE
print(last_data)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:105: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:105: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, return_sequences=True, input_shape=(None, 1))`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:111: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`


Training ------------
Epoch 1/1
50/50 [==============================] - 0s - loss: 4.6804      
[[ 4.67653275]]


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


[ 5.22083497]
